In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()

In [3]:
x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full)

In [5]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

In [6]:
x_train_A, x_train_B = x_train[:, :5], x_train[:,2:]
x_valid_A, x_valid_B = x_valid[:, :5], x_valid[:, 2:]
x_test_A, x_test_B = x_test[:,  :5], x_test[:, 2:]

In [7]:
import tensorflow as tf
from tensorflow import keras

In [8]:
input_A = keras.layers.Input(shape=[5],name='wide_input')
input_B = keras.layers.Input(shape=[6],name='deep_input')
hidden1 = keras.layers.Dense(30,activation='relu')(input_B)
hidden2 = keras.layers.Dense(30,activation='relu')(hidden1)
concat = keras.layers.concatenate([input_A,hidden2])
output = keras.layers.Dense(1, name='output')(concat)

model = keras.Model(inputs=[input_A, input_B], outputs = [output])

In [9]:
## visualizing the model
## visualizing the created model
keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [10]:
model.compile(loss='mse', optimizer='Adam')

In [11]:
## creating callbacks (ModelCheckpoint and EarlyStopping)
checkpoint_cb = keras.callbacks.ModelCheckpoint('my_keras_model.h5',save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)

In [12]:
## creating tensorboard callback

import os
root_logdir = os.path.join(os.curdir, 'my_logs')

def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d_%H_%M_%S')
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [13]:
## we can give epoch a big value since anyway a training will be stopped if there is no progress for 10 consecutive epochs
history = model.fit((x_train_A, x_train_B), y_train, epochs=100, validation_data=((x_valid_A,x_valid_B),y_valid), 
                           callbacks=[checkpoint_cb,early_stopping_cb,tensorboard_cb])

Epoch 1/100
363/363 [==============================] - 2s 3ms/step - loss: 1.7425 - val_loss: 0.6796
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.6239 - val_loss: 0.4471
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4328 - val_loss: 0.4127
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4088 - val_loss: 0.3900
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3899 - val_loss: 0.3792
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3782 - val_loss: 0.3687
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3673 - val_loss: 0.3678
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3624 - val_loss: 0.3562
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3639 - val_loss: 0.3520
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3550 - val_lo

In [14]:
mse_test = model.evaluate((x_test_A, x_test_B), y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.3177


In [15]:
x_new_A = x_test_A[:3]
x_new_B = x_test_B[:3]

In [16]:
y_pred = model.predict((x_new_A,x_new_B))

1/1 [==============================] - 0s 96ms/step


In [17]:
y_pred

array([[1.9664652],
       [1.5322871],
       [2.4195929]], dtype=float32)

In [18]:
## opening tensorboard server
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

Reusing TensorBoard on port 6006 (pid 25804), started 0:03:18 ago. (Use '!kill 25804' to kill it.)